In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import dtw
from keras.preprocessing.sequence import pad_sequences

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv("/Users/a123/Desktop/CMU/Capstone/CRS-Detection/Datasets/final_data_clipped.csv")
df.head(5)

/var/folders/9q/_q_h4th94vj56dq6c3yv3gj80000gn/T/ipykernel_65832/1396069146.py:1: DtypeWarning: Columns (38,51,53,54,84,108,122,139,142,159,160,161) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/a123/Desktop/CMU/Capstone/CRS-Detection/Datasets/final_data_clipped.csv")


,Unnamed: 0,patient_id,pr_display,spo2_display,resp_display,pulse_rate_obscount,pulse_rate_avg,pulse_rate_min,pulse_rate_max,pulse_rate_iqr,...,QC Deviation from median.1,Weight at time of infusion,precise_datetime,CRS intervention,CRS Grade (nurse),Temperature,SpO2,BP,HR,Temperature_C
0,0,7001,71 beats/min,96%,17%,10.0,70.01,66.63,71.52,2.88,...,0.047985,74.325238,2022-10-21 08:00:00.000000000,Tocilizumab x 1 on 1/22/24,1.0,100.9,WNL,WNL,WNL,38.277778
1,1,7001,71 beats/min,96%,17%,10.0,70.01,66.63,71.52,2.88,...,0.047985,74.325238,2022-10-21 08:00:00.000000000,Tocilizumab x 1 on 1/22/24,NaN,NaN,NaN,NaN,NaN,NaN
2,2,7001,71 beats/min,96%,17%,10.0,70.01,66.63,71.52,2.88,...,0.048813,74.325238,2022-10-21 08:14:32.727272704,Tocilizumab x 1 on 1/22/24,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7001,71 beats/min,96%,17%,10.0,70.01,66.63,71.52,2.88,...,0.049641,74.325238,2022-10-21 08:29:05.454545408,Tocilizumab x 1 on 1/22/24,NaN,NaN,NaN,NaN,NaN,NaN
4,4,7001,71 beats/min,96%,17%,10.0,70.01,66.63,71.52,2.88,...,0.050469,74.325238,2022-10-21 08:43:38.181818112,Tocilizumab x 1 on 1/22/24,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df[(df.patient_id == 7005)&(df['CRS on date (0 No, 1 Yes)']==1)][['patient_id',"CRS on date (0 No, 1 Yes)",'datetime','Agent (JNJ/BMS/Caribou)']]

,patient_id,"CRS on date (0 No, 1 Yes)",datetime,Agent (JNJ/BMS/Caribou)
64025,7005,1,2022-12-14 20:01:00,JNJ OOS
64026,7005,1,2022-12-14 20:02:00,JNJ OOS
64027,7005,1,2022-12-14 20:03:00,JNJ OOS
64028,7005,1,2022-12-14 20:04:00,JNJ OOS
64029,7005,1,2022-12-14 20:05:00,JNJ OOS
...,...,...,...,...
65009,7005,1,2022-12-15 19:56:00,JNJ OOS
65010,7005,1,2022-12-15 19:57:00,JNJ OOS
65011,7005,1,2022-12-15 19:58:00,JNJ OOS
65012,7005,1,2022-12-15 19:59:00,JNJ OOS


In [4]:
list(df.columns)

['Unnamed: 0',
 'patient_id',
 'pr_display',
 'spo2_display',
 'resp_display',
 'pulse_rate_obscount',
 'pulse_rate_avg',
 'pulse_rate_min',
 'pulse_rate_max',
 'pulse_rate_iqr',
 'spo2_obscount',
 'spo2_avg',
 'spo2_min',
 'spo2_max',
 'respiratory_rate_obscount',
 'respiratory_rate_avg',
 'respiratory_rate_min',
 'respiratory_rate_max',
 'covered_skin_temperature_obscount',
 'covered_skin_temperature_avg',
 'covered_skin_temperature_min',
 'covered_skin_temperature_max',
 'covered_axil_temperature_obscount',
 'covered_axil_temperature_avg',
 'covered_axil_temperature_min',
 'covered_axil_temperature_max',
 'motion_obscount',
 'motion_avg',
 'step_count_sum',
 'step_count_obscount',
 'time',
 'datetime',
 'PT_ID',
 'Sample ID',
 'Agent (JNJ/BMS/Caribou)',
 'Date of CAR-T infusion',
 'Age',
 'Date of Sample',
 'Days in relation to CAR-T  infusion',
 'Patient experienced CRS at any point (Yes/No)',
 'CRS on date (0 No, 1 Yes)',
 'CRS Grade ',
 'ICANs on date (0 -no, 1 yes)',
 'TOCI give

In [5]:
complete_set = ['PT_ID','CRS on date (0 No, 1 Yes)','Agent (JNJ/BMS/Caribou)','datetime', 'spo2_avg', 'pulse_rate_avg','respiratory_rate_avg', 'covered_skin_temperature_avg','covered_axil_temperature_avg','Highest Ferritin',
 'Highest CRP','IL8',
 'TNFRSF9',
 'TIE2',
 'MCP-3',
 'CD40-L',
 'IL-1 alpha',
 'CD244',
 'EGF',
 'ANGPT1',
 'IL7',
 'PGF',
 'IL6',
 'ADGRG1',
 'MCP-1',
 'CRTAM',
 'CXCL11',
 'MCP-4',
 'TRAIL',
 'FGF2',
 'CXCL9',
 'CD8A',
 'CAIX',
 'MUC-16',
 'ADA',
 'CD4',
 'NOS3',
 'IL2',
 'Gal-9',
 'VEGFR-2',
 'CD40',
 'IL18',
 'GZMH',
 'KIR3DL1',
 'LAP TGF-beta-1',
 'CXCL1',
 'TNFSF14',
 'IL33',
 'TWEAK',
 'PDGF subunit B',
 'PDCD1',
 'FASLG',
 'CD28',
 'CCL19',
 'MCP-2',
 'CCL4',
 'IL15',
 'Gal-1',
 'PD-L1',
 'CD27',
 'CXCL5',
 'IL5',
 'HGF',
 'GZMA',
 'HO-1',
 'CX3CL1',
 'CXCL10',
 'CD70',
 'IL10',
 'TNFRSF12A',
 'CCL23',
 'CD5',
 'CCL3',
 'MMP7',
 'ARG1',
 'NCR1',
 'DCN',
 'TNFRSF21',
 'TNFRSF4',
 'MIC-A/B',
 'CCL17',
 'ANGPT2',
 'PTN',
 'CXCL12',
 'IFN-gamma',
 'LAMP3',
 'CASP-8',
 'ICOSLG',
 'MMP12',
 'CXCL13',
 'PD-L2',
 'VEGFA',
 'IL4',
 'LAG3',
 'IL12RB1',
 'IL13',
 'CCL20',
 'TNF',
 'KLRD1',
 'GZMB',
 'CD83',
 'IL12',
 'CSF-1',]

In [6]:
columns_test_1 = ['PT_ID','CRS on date (0 No, 1 Yes)','Agent (JNJ/BMS/Caribou)','datetime', 'spo2_avg', 'pulse_rate_avg','respiratory_rate_avg', 'covered_skin_temperature_avg','IL8','IL6','CXCL10','IFN-gamma','CCL23', 'CASP-8',
 'CXCL13'] 
df_subset = df[columns_test_1]
df_subset.head(5)

,PT_ID,"CRS on date (0 No, 1 Yes)",Agent (JNJ/BMS/Caribou),datetime,spo2_avg,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,CXCL10,IFN-gamma,CCL23,CASP-8,CXCL13
0,7001,0,BMS,2022-01-21 23:30:00,97.83,70.01,24.35,27.650,5.570850,4.754540,9.135090,5.938620,10.893300,4.387440,6.721580
1,7001,1,BMS,2022-10-21 16:02:00,97.83,70.01,24.35,27.650,5.570850,4.754540,9.135090,5.938620,10.893300,4.387440,6.721580
2,7001,1,BMS,2022-10-21 16:03:00,97.83,70.01,19.86,27.735,5.575702,4.773422,9.158341,6.003697,10.897528,4.403390,6.724164
3,7001,1,BMS,2022-10-21 17:42:00,97.83,70.01,15.37,27.820,5.580553,4.792303,9.181591,6.068774,10.901756,4.419340,6.726747
4,7001,1,BMS,2022-10-21 17:43:00,97.83,70.01,11.77,27.780,5.585405,4.811185,9.204842,6.133852,10.905983,4.435289,6.729331


In [7]:
df_JNJ = df_subset[(df_subset['Agent (JNJ/BMS/Caribou)']=='JNJ') | (df_subset['Agent (JNJ/BMS/Caribou)']=='JNJ OOS')]
df_BMS = df_subset[(df_subset['Agent (JNJ/BMS/Caribou)']=='BMS')]

In [8]:
# Individualize the data
# Create a baseline dataframe
data = df_JNJ.copy()

baseline = data.groupby('PT_ID').first().reset_index()

# Subtract the baseline values for numeric columns
columns_to_individualize = [col for col in columns_test_1 if col not in ['PT_ID', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)','datetime']]

for col in columns_to_individualize:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col] = data[col] - data.groupby('PT_ID')[col].transform('first')

data.head()


,PT_ID,"CRS on date (0 No, 1 Yes)",Agent (JNJ/BMS/Caribou),datetime,spo2_avg,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,CXCL10,IFN-gamma,CCL23,CASP-8,CXCL13
42639,7004,0,JNJ,2022-12-05 15:44:00,0.0,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
42640,7004,0,JNJ,2022-12-05 15:45:00,0.0,0.0,0.000,0.00,0.002603,-0.000316,-0.001205,-0.000085,-0.000287,-0.005952,-0.000051
42641,7004,0,JNJ,2022-12-05 15:46:00,0.0,0.0,7.210,0.01,0.005205,-0.000633,-0.002410,-0.000171,-0.000574,-0.011904,-0.000102
42642,7004,0,JNJ,2022-12-05 15:47:00,0.0,0.0,4.605,0.08,0.007808,-0.000949,-0.003616,-0.000256,-0.000861,-0.017857,-0.000153
42643,7004,0,JNJ,2022-12-05 15:48:00,0.0,0.0,2.000,0.02,0.010410,-0.001265,-0.004821,-0.000342,-0.001148,-0.023809,-0.000203


In [9]:
data[(data.PT_ID == 7005)&(data['CRS on date (0 No, 1 Yes)']==1)][["CRS on date (0 No, 1 Yes)",'datetime']]

,"CRS on date (0 No, 1 Yes)",datetime
64025,1,2022-12-14 20:01:00
64026,1,2022-12-14 20:02:00
64027,1,2022-12-14 20:03:00
64028,1,2022-12-14 20:04:00
64029,1,2022-12-14 20:05:00
...,...,...
65009,1,2022-12-15 19:56:00
65010,1,2022-12-15 19:57:00
65011,1,2022-12-15 19:58:00
65012,1,2022-12-15 19:59:00


In [10]:
data[(data.PT_ID == 7005)&(pd.to_datetime(data['datetime'])>pd.to_datetime("2022-12-14 21:05:00"))][["CRS on date (0 No, 1 Yes)",'datetime']]

,"CRS on date (0 No, 1 Yes)",datetime
64086,1,2022-12-14 23:27:00
64087,1,2022-12-14 23:28:00
64088,1,2022-12-14 23:29:00
64089,1,2022-12-14 23:30:00
64090,1,2022-12-14 23:31:00
...,...,...
69885,0,2022-12-20 15:53:00
69886,0,2022-12-20 15:54:00
69887,0,2022-12-20 15:55:00
69888,0,2022-12-20 15:56:00


In [11]:
# Define the window size for rolling statistics (e.g., past 6 hours)
window_size = 6
lag_size = 6
time_interval = 30

# Sort data by patient ID and datetime
data = data.sort_values(by=['PT_ID', 'datetime'])

# Function to create lagged features and rolling statistics
def add_past_features(data, columns,lag_size):
    for col in columns:
        # Add lagged values 
        for lag_base in range(1, lag_size + 1):
            lag = lag_base * time_interval
            data[f'{col}_lag_{lag}'] = data.groupby('PT_ID')[col].shift(lag)
        
        rolling_size = lag_size * time_interval
        # Add rolling statistics
        data[f'{col}_rolling_mean_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).mean().reset_index(level=0, drop=True)
        data[f'{col}_rolling_std_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).std().reset_index(level=0, drop=True)
        data[f'{col}_rolling_min_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).min().reset_index(level=0, drop=True)
        data[f'{col}_rolling_max_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).max().reset_index(level=0, drop=True)


    return data

# Add past features for selected columns
columns_to_process = [col for col in columns_test_1 if col not in ['PT_ID','CRS on date (0 No, 1 Yes)','Agent (JNJ/BMS/Caribou)','datetime']]
data = add_past_features(data, columns_to_process, lag_size)


/var/folders/9q/_q_h4th94vj56dq6c3yv3gj80000gn/T/ipykernel_65832/2548389237.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_rolling_std_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).std().reset_index(level=0, drop=True)
/var/folders/9q/_q_h4th94vj56dq6c3yv3gj80000gn/T/ipykernel_65832/2548389237.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_rolling_min_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).min().reset_index(leve

In [12]:
data.shape

(173156, 125)

In [13]:
from datetime import timedelta

def assign_crs_in_6_hours(data):
    """
    Assign CRS_in_6_hours for each row based on whether `datetime + 6 hours` falls within a CRS occurrence time frame.

    Parameters:
        data (DataFrame): Input DataFrame with 'PT_ID', 'datetime', and 'CRS on date (0 No, 1 Yes)' columns.

    Returns:
        DataFrame: Updated DataFrame with a new column 'CRS_in_6_hours'.
    """
    # Ensure 'datetime' is a datetime object
    data['datetime'] = pd.to_datetime(data['datetime'])
    data = data.sort_values(by=['PT_ID', 'datetime'])

    # Initialize a new column
    data['CRS_in_6_hours'] = 0

    # Process each patient group separately
    for pt_id, group in data.groupby('PT_ID'):
        # Sort by datetime for the current patient
        group = group.sort_values('datetime')

        # Identify CRS occurrence start and end timeframes
        crs_start = group.index[(group['CRS on date (0 No, 1 Yes)'].shift(1) == 0) &
                                (group['CRS on date (0 No, 1 Yes)'] == 1)].tolist()
        crs_end = group.index[(group['CRS on date (0 No, 1 Yes)'].shift(1) == 1) &
                              (group['CRS on date (0 No, 1 Yes)'] == 0)].tolist()

        # If a CRS event starts but does not end, assume it continues until the last datetime
        if len(crs_start) > len(crs_end):
            crs_end.append(group.index[-1])

        # Assign CRS_in_6_hours for each row
        for start_idx, end_idx in zip(crs_start, crs_end):
            crs_start_time = group.loc[start_idx, 'datetime']
            crs_end_time = group.loc[end_idx, 'datetime']
            
            # Any datetime + 6 hours within the CRS occurrence timeframe is set to 1
            within_crs_timeframe = (group['datetime'] + timedelta(hours=6) >= crs_start_time) & \
                                   (group['datetime'] + timedelta(hours=6) <= crs_end_time)
            data.loc[group[within_crs_timeframe].index, 'CRS_in_6_hours'] = 1

    return data

# Apply the function to your dataset
data = assign_crs_in_6_hours(data)


In [14]:
data.shape

(173156, 126)

In [15]:
data[data['CRS_in_6_hours']==1]

,PT_ID,"CRS on date (0 No, 1 Yes)",Agent (JNJ/BMS/Caribou),datetime,spo2_avg,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,...,CXCL13_lag_60,CXCL13_lag_90,CXCL13_lag_120,CXCL13_lag_150,CXCL13_lag_180,CXCL13_rolling_mean_180,CXCL13_rolling_std_180,CXCL13_rolling_min_180,CXCL13_rolling_max_180,CRS_in_6_hours
63665,7005,0,JNJ OOS,2022-12-14 14:01:00,-2.255,33.12,-1.07,5.76,0.889360,1.249300,...,0.801890,0.801890,0.801890,0.801890,0.801890,0.801890,0.000000,0.801890,0.801890,1
63666,7005,0,JNJ OOS,2022-12-14 14:02:00,-1.820,34.77,1.08,5.77,0.889360,1.249300,...,0.801890,0.801890,0.801890,0.801890,0.801890,0.801890,0.000000,0.801890,0.801890,1
63667,7005,0,JNJ OOS,2022-12-14 14:03:00,-1.820,35.61,3.29,5.79,0.889360,1.249300,...,0.801890,0.801890,0.801890,0.801890,0.801890,0.801890,0.000000,0.801890,0.801890,1
63668,7005,0,JNJ OOS,2022-12-14 14:04:00,-2.820,33.46,-1.34,5.79,0.889360,1.249300,...,0.801890,0.801890,0.801890,0.801890,0.801890,0.801890,0.000000,0.801890,0.801890,1
63669,7005,0,JNJ OOS,2022-12-14 14:05:00,-1.820,32.59,0.79,5.79,0.889360,1.249300,...,0.801890,0.801890,0.801890,0.801890,0.801890,0.801890,0.000000,0.801890,0.801890,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313952,7028,1,JNJ,2023-10-06 01:57:00,7.810,-71.03,4.07,0.24,1.956020,3.989555,...,0.361414,0.360956,0.360498,0.360041,0.359583,0.360964,0.000795,0.359599,0.362329,1
313953,7028,1,JNJ,2023-10-06 01:58:00,8.800,-71.32,2.84,0.28,1.957046,3.991666,...,0.361429,0.360971,0.360514,0.360056,0.359599,0.360979,0.000795,0.359614,0.362344,1
313954,7028,1,JNJ,2023-10-06 01:59:00,8.380,-72.11,6.42,0.27,1.958072,3.993777,...,0.361444,0.360987,0.360529,0.360071,0.359614,0.360994,0.000795,0.359629,0.362359,1
313955,7028,1,JNJ,2023-10-06 02:00:00,5.730,-70.87,3.42,0.27,1.959099,3.995888,...,0.361459,0.361002,0.360544,0.360087,0.359629,0.361009,0.000795,0.359644,0.362374,1


In [16]:
data[data['CRS_in_6_hours']==1].head(10)

,PT_ID,"CRS on date (0 No, 1 Yes)",Agent (JNJ/BMS/Caribou),datetime,spo2_avg,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,...,CXCL13_lag_60,CXCL13_lag_90,CXCL13_lag_120,CXCL13_lag_150,CXCL13_lag_180,CXCL13_rolling_mean_180,CXCL13_rolling_std_180,CXCL13_rolling_min_180,CXCL13_rolling_max_180,CRS_in_6_hours
63665,7005,0,JNJ OOS,2022-12-14 14:01:00,-2.255,33.12,-1.07,5.76,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63666,7005,0,JNJ OOS,2022-12-14 14:02:00,-1.820,34.77,1.08,5.77,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63667,7005,0,JNJ OOS,2022-12-14 14:03:00,-1.820,35.61,3.29,5.79,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63668,7005,0,JNJ OOS,2022-12-14 14:04:00,-2.820,33.46,-1.34,5.79,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63669,7005,0,JNJ OOS,2022-12-14 14:05:00,-1.820,32.59,0.79,5.79,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63670,7005,0,JNJ OOS,2022-12-14 14:06:00,-1.820,33.36,3.35,5.80,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63671,7005,0,JNJ OOS,2022-12-14 14:07:00,-2.145,34.23,0.72,5.81,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63672,7005,0,JNJ OOS,2022-12-14 14:08:00,-2.470,34.13,2.10,5.81,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63673,7005,0,JNJ OOS,2022-12-14 14:09:00,-2.430,33.59,4.45,5.83,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1
63674,7005,0,JNJ OOS,2022-12-14 14:10:00,-2.390,34.58,1.07,5.84,0.88936,1.2493,...,0.80189,0.80189,0.80189,0.80189,0.80189,0.80189,0.0,0.80189,0.80189,1


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Ensure datetime is in datetime format
data['datetime'] = pd.to_datetime(data['datetime'])

# Calculate 0.8 quantile cutoff for each patient
quantile_cutoffs = data.groupby('PT_ID')['datetime'].quantile(0.8).reset_index()
quantile_cutoffs.columns = ['PT_ID', 'cutoff_date']

# Merge the cutoff dates back into the original dataframe
data = data.merge(quantile_cutoffs, on='PT_ID')

# Create train and test datasets based on the cutoff date
train_data = data[data['datetime'] < data['cutoff_date']]
test_data = data[data['datetime'] >= data['cutoff_date']]

# Separate features and target
X_train = train_data.drop(columns=['PT_ID', 'datetime', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'CRS_in_6_hours', 'cutoff_date'])
y_train = train_data['CRS_in_6_hours']

X_test = test_data.drop(columns=['PT_ID', 'datetime', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'CRS_in_6_hours', 'cutoff_date'])
y_test = test_data['CRS_in_6_hours']



In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, classification_report

# Convert data to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train  # Target variable in the training set
)

# Set the scale_pos_weight parameter in XGBoost
scale_pos_weight = class_weights[1] / class_weights[0]

# XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'scale_pos_weight': scale_pos_weight  # Add the class imbalance weight
}

# Train the XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
model = xgb.train(params, dtrain, num_boost_round=100)


# Make predictions
y_pred = (model.predict(dtest) > 0.49).astype(int)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# Predict probabilities for AUC-ROC
y_prob = model.predict(dtest)
auc = roc_auc_score(y_test, y_prob)
print("AUC-ROC Score:", auc)



In [ ]:
from sklearn.metrics import f1_score

def find_best_threshold_class1(y_true, y_prob, step=0.05):
    """
    Find the best threshold for maximizing F1-score for class 1 (minority class).
    
    Parameters:
        y_true (array-like): True labels (binary: 0 or 1).
        y_prob (array-like): Predicted probabilities for the positive class.
        step (float): Step size for threshold search. Default is 0.05.
    
    Returns:
        best_threshold (float): Threshold that gives the highest F1-score for class 1.
        best_f1 (float): The highest F1-score for class 1 achieved.
    """
    best_threshold = 0.0
    best_f1 = 0.0  # Start with the lowest possible F1-score

    # Iterate through thresholds from 0 to 1
    for threshold in np.arange(0, 1 + step, step):
        y_pred = (y_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred, pos_label=1)  # F1 for class 1
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1
best_threshold, best_f1 = find_best_threshold_class1(y_test, y_prob, step=0.01)
best_threshold

In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Plot feature importance
plt.figure(figsize=(10, 8))
xgb.plot_importance(model, max_num_features=20)  # Show top 20 features
plt.title('Feature Importance')
plt.xlabel('F-Score')
plt.ylabel('Features')
plt.tight_layout()
plt.show()


In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import xgboost as xgb

# Step 2: Apply Random Oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Check the class distribution after oversampling
print("Class distribution after oversampling:")
print(pd.Series(y_train_resampled).value_counts())

# Step 3: Convert to XGBoost DMatrix
dtrain = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
dtest = xgb.DMatrix(X_test, label=y_test)

# Step 4: Define XGBoost Parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Step 5: Train the XGBoost Model
model = xgb.train(params, dtrain, num_boost_round=100)

# Step 6: Predict and Evaluate
y_prob = model.predict(dtest)  # Predicted probabilities
y_pred = (y_prob > 0.68).astype(int)  # Convert probabilities to binary predictions

print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC Score:", roc_auc_score(y_test, y_prob))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
best_threshold, best_f1 = find_best_threshold_class1(y_test, y_prob, step=0.01)
best_threshold

In [ ]:
X_train_resampled.dtypes


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Plot feature importance
plt.figure(figsize=(10, 8))
xgb.plot_importance(model, max_num_features=20)  # Show top 20 features
plt.title('Feature Importance')
plt.xlabel('F-Score')
plt.ylabel('Features')
plt.tight_layout()
plt.show()


### Patients split

In [27]:
unique_patients = data['PT_ID'].unique()

# Split patients into 80% train and 20% test
train_patients, test_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

# Assign data to train and test sets based on the split
train_data = data[data['PT_ID'].isin(train_patients)]
test_data = data[data['PT_ID'].isin(test_patients)]

def filter_test_data(test_data):
    """
    Filter the test data to include only rows from 6 hours before CRS occurrence
    (where 'CRS on date (0 No, 1 Yes)' changes from 0 to 1) until the CRS occurrence.
    
    Parameters:
        test_data (DataFrame): The test dataset including 'PT_ID', 'datetime', and 'CRS on date (0 No, 1 Yes)'.
    
    Returns:
        DataFrame: Filtered test data.
    """
    filtered_data = []

    # Group by patient
    for pt_id, group in test_data.groupby('PT_ID'):
        group = group.sort_values('datetime')  # Ensure chronological order
        
        # Find transitions from 0 to 1 in 'CRS on date (0 No, 1 Yes)'
        transition_indices = group.index[(group['CRS on date (0 No, 1 Yes)'].shift(1) == 0) & 
                                         (group['CRS on date (0 No, 1 Yes)'] == 1)]

        for idx in transition_indices:
            # Time of CRS occurrence
            crs_time = group.loc[idx, 'datetime']
            
            # Calculate the 6-hour window
            # start_time = crs_time - timedelta(hours=6)
            start_time = group['datetime'].min()
            
            # Filter rows within the 6-hour window
            filtered_group = group[(group['datetime'] >= start_time) & (group['datetime'] <= crs_time)]
            filtered_data.append(filtered_group)

    # Concatenate all filtered groups
    return pd.concat(filtered_data)

# Apply the filtering function to the test data
filtered_test_data = filter_test_data(test_data)



In [ ]:
filtered_test_data

In [ ]:
filtered_test_data[filtered_test_data['CRS_in_6_hours']==0][['PT_ID','datetime','CRS on date (0 No, 1 Yes)','CRS_in_6_hours']]

In [ ]:
filtered_test_data[(filtered_test_data.PT_ID==7005) & (filtered_test_data['CRS on date (0 No, 1 Yes)'])==0][['datetime','CRS on date (0 No, 1 Yes)','CRS_in_6_hours']]

In [31]:
# Separate features and target
X_train = train_data.drop(columns=['PT_ID', 'datetime', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'CRS_in_6_hours', 'cutoff_date'])
y_train = train_data['CRS_in_6_hours']

X_test = filtered_test_data.drop(columns=['PT_ID', 'datetime', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'CRS_in_6_hours', 'cutoff_date'])
y_test = filtered_test_data['CRS_in_6_hours']

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, classification_report
# Step 2: Apply Random Oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Check the class distribution after oversampling
print("Class distribution after oversampling:")
print(pd.Series(y_train_resampled).value_counts())

# Step 3: Convert to XGBoost DMatrix
dtrain = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
dtest = xgb.DMatrix(X_test, label=y_test)

# Step 4: Define XGBoost Parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Step 5: Train the XGBoost Model
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
y_pred = (model.predict(dtest) > 0.68).astype(int)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# Predict probabilities for AUC-ROC
y_prob = model.predict(dtest)
#auc = roc_auc_score(y_test, y_prob)
print("AUC-ROC Score:", auc)



In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, classification_report

# Convert data to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train  # Target variable in the training set
)

# Set the scale_pos_weight parameter in XGBoost
scale_pos_weight = class_weights[1] / class_weights[0]

# XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'scale_pos_weight': scale_pos_weight  # Add the class imbalance weight
}

# Train the XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
model = xgb.train(params, dtrain, num_boost_round=100)


# Make predictions
y_pred = (model.predict(dtest) > 0.49).astype(int)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# Predict probabilities for AUC-ROC
y_prob = model.predict(dtest)
#auc = roc_auc_score(y_test, y_prob)
print("AUC-ROC Score:", auc)



### Model Finetuning

In [34]:
from sklearn.metrics import f1_score

def evaluate_model_with_weights(X_train, X_test, y_train, y_test, selected_features, scale_pos_weight):
    # Create XGBoost DMatrix for the selected features
    dtrain = xgb.DMatrix(X_train[selected_features], label=y_train)
    dtest = xgb.DMatrix(X_test[selected_features], label=y_test)

    # Define XGBoost parameters (include scale_pos_weight)
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': 6,
        'learning_rate': 0.1,
        'scale_pos_weight': scale_pos_weight,
    }

    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=100)

    # Predict probabilities and calculate F1-score for the minority class
    y_prob = model.predict(dtest)
    y_pred = (y_prob > 0.5).astype(int)
    f1 = f1_score(y_test, y_pred, pos_label=1)  # F1-score for minority class
    return f1


In [35]:
# from itertools import combinations
# import numpy as np

# # Define all available features
# all_features = X_train.columns.tolist()

# # Initialize variables to store the best combination and its score
# best_features = None
# best_f1 = -np.inf

# # Iterate over feature subsets
# for r in range(1, len(all_features) + 1):  # From 1 feature to all features
#     for subset in combinations(all_features, r):
#         # Evaluate the model with the current subset of features
#         f1 = evaluate_model_with_weights(
#             X_train=X_train,
#             X_test=X_test,
#             y_train=y_train,
#             y_test=y_test,
#             selected_features=list(subset),
#             scale_pos_weight=scale_pos_weight
#         )
        
#         # Update the best combination if the current subset performs better
#         if f1 > best_f1:
#             best_f1 = f1
#             best_features = subset

# # Display the best feature subset and its F1-score
# print(f"Best Features: {best_features}")
# print(f"Best F1-Score: {best_f1}")


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

# Define the feature selection method
def recursive_feature_elimination(X_train, X_test, y_train, y_test, scale_pos_weight, step=1):
    """
    Perform Recursive Feature Elimination (RFE) to select the most relevant features.
    """
    # Initialize the XGBoost model
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': 6,
        'learning_rate': 0.1,
        'scale_pos_weight': scale_pos_weight,
    }
    
    model = xgb.XGBClassifier(**params)

    # Use RFE to select features
    rfe = RFE(estimator=model, step=step)
    rfe = rfe.fit(X_train, y_train)

    # Get the selected features
    selected_features = X_train.columns[rfe.support_]

    # Evaluate model with selected features
    dtrain = xgb.DMatrix(X_train[selected_features], label=y_train)
    dtest = xgb.DMatrix(X_test[selected_features], label=y_test)

    final_model = xgb.train(params, dtrain, num_boost_round=100)
    y_prob = final_model.predict(dtest)
    y_pred = (y_prob > 0.49).astype(int)

    # Calculate F1-score for minority class
    f1 = f1_score(y_test, y_pred, pos_label=1)
    return selected_features, f1

# Apply RFE
selected_features, best_f1 = recursive_feature_elimination(X_train, X_test, y_train, y_test, scale_pos_weight)
print(f"Selected Features: {selected_features}")
print(f"Best F1-Score: {best_f1}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assume `best_features` contains the list of selected features from the feature selection process

# Filter the training and testing data to include only the best features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Calculate class weights for the training data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train  # Target variable in the training set
)
scale_pos_weight = class_weights[1] / class_weights[0]  # Class imbalance adjustment

# Define XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'scale_pos_weight': scale_pos_weight,  # Handle class imbalance
}

# Train the model with selected features
dtrain = xgb.DMatrix(X_train_selected, label=y_train)
dtest = xgb.DMatrix(X_test_selected, label=y_test)

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions with the new model
y_prob = model.predict(dtest)
y_pred = (y_prob > 0.96).astype(int)  # Threshold can be adjusted as needed

# Evaluate the new model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Calculate AUC-ROC
auc = roc_auc_score(y_test, y_prob)
print("AUC-ROC Score:", auc)


In [ ]:
best_threshold, best_f1 = find_best_threshold_class1(y_test, y_prob, step=0.01)
best_threshold

In [ ]:
best_f1